In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Préliminaires

## Lecture et description du fichier

In [ ]:
file_path = 'default-of-credit-card-clients-dataset.zip'
target    = 'default.payment.next.month'

df = pd.read_csv(filepath_or_buffer=file_path)
print(df.shape)
df = df.drop('ID', axis=1)
print(df[target].value_counts() / len(df))
df.head()

## Type des features

Parcourir chaque colonne du fichier

- Identifier le type
- Compter le nombre de valeurs uniques N_u
    - Si numérique :
        - Si N_u < 10 : variable catégorielle
        - Si N_u >= 10 : à investiguer
        - Si N_u > 50 : variable continue
    - Si character :
        - Si N_u < 10 : variable catégorielle
        - Si N_u >= 10 : à investiguer
        - Si N_u > 50 : variable textuelle

In [ ]:
raw_features  = df.drop(target, axis=1).columns

categorical_features = []
numerical_features = []
other_features = []
to_investigate = []

for feature in raw_features:
    N_u = len(df[feature].unique())
    feature_type = df[feature].dtype
    if feature_type == np.float64 or feature_type == np.int64:
        if(N_u) < 10:
            categorical_features.append([feature, N_u])
        if(N_u >= 10 and N_u <50):
            to_investigate.append([feature, N_u])
        if(N_u >= 50):
            numerical_features.append([feature, N_u])
    else:
        other_features.append([feature, N_u])
            
        
print("categorical features (<10 unique values):{}".format(categorical_features))
print("features to investigate :{}".format(to_investigate))
print("numerical features (more than 50 unique values):{}".format(numerical_features))
print("other features :{}".format(other_features))

    

In [ ]:
features_to_categorize = ['SEX', 'EDUCATION', 'MARRIAGE', 
                          'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
for feature in features_to_categorize:
    df[feature] = df[feature].astype('category')

In [ ]:
df.describe(percentiles=[.2, .4, .6, .8]).transpose()

In [ ]:
df.select_dtypes(include=['category']).describe().transpose()

In [ ]:
# Making correlation coefficients pair plot of all feature in order to identify degenrate features
plt.figure(figsize=(25,25))
ax = plt.axes()
corr = df.corr()
sns.heatmap(corr, vmax=1,vmin=-1, square=True, annot=True, cmap='Spectral',linecolor="white", linewidths=0.01, ax=ax)
ax.set_title('Correlation Coefficient Pair Plot',fontweight="bold", size=30)
plt.show()

## Analyse descriptive

Le code suivant génère 2 fichiers pdf :
- numerical_features_plots.pdf : les distributions de chaque variable numérique conditionnellement à la cible
- categorical_features_plots.pdf : les barplots des variables categorielles

In [ ]:
df.select_dtypes(np.number).columns

In [ ]:
df.select_dtypes('category').columns

In [ ]:
f, ax = plt.subplots(nrows=1, figsize=(20, 12))

AGE_bin = pd.qcut(df['AGE'], 10, duplicates='drop', labels=None)
LIMIT_BAL_bin = pd.qcut(df['LIMIT_BAL'], 10, duplicates='drop', labels=None)
PAY_AMT1_bin = pd.qcut(df['PAY_AMT1'], 10, duplicates='drop', labels=None)

sns.barplot(x = PAY_AMT1_bin,
            y = df['BILL_AMT1'],
            hue = df[target])

In [ ]:
# Variables numériques
# Génère un fichier pdf

from matplotlib.backends.backend_pdf import PdfPages
pp = PdfPages('numerical_features_plots.pdf')

for feature in df.select_dtypes(np.number).columns:
    sns.distplot(df[feature][df[target] == 1])
    sns.distplot(df[feature][df[target] == 0])
    pp.savefig()
    plt.show()
    
pp.close()

In [ ]:
# Variables catégorielles
sns.set_style("whitegrid")
pp = PdfPages('categorical_features_plots.pdf')

for feature in df.select_dtypes('category').columns:
    crs_tab = pd.crosstab(df[feature], df[target]).stack().reset_index().rename(columns={0:'value'})
    sns.barplot(x=crs_tab[feature], y=crs_tab.value, hue=crs_tab[target])
    pp.savefig()
    plt.show()
    
pp.close()

In [ ]:
sns.pairplot(df[['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', target]], hue=target)

In [ ]:
df[['BILL_AMT1', 'PAY_AMT1', 'PAY_0', 'BILL_AMT2', 'PAY_AMT2', 'PAY_2']]

## Représentation T-SNE

In [ ]:
from sklearn.manifold import TSNE
from matplotlib.ticker import NullFormatter

(fig, subplots) = plt.subplots(ncols=4, figsize=(15, 8))

n_components = 2
perplexities = [5, 30, 50, 100]

df_samp = df.sample(frac=0.2)

red = df_samp[target] == 0
green = df_samp[target] == 1



for i, perplexity in enumerate(perplexities):
    ax = subplots[i]
    ax.set_title("Perplexity=%d" % perplexity)
    
    tsne = TSNE(n_components=n_components, init='random', random_state=0, perplexity=perplexity)
    Y = tsne.fit_transform(df_samp)
    
    ax.scatter(Y[red, 0], Y[red, 1], c="r")
    ax.scatter(Y[green, 0], Y[green, 1], c="g")
    ax.xaxis.set_major_formatter(NullFormatter())
    ax.yaxis.set_major_formatter(NullFormatter())
    ax.axis('tight')


## Transfo des données

Données catégorielles sont transformées en 0/1
Données numériques transformées 

- f:x => sig(x)*log(1+abs(x))

In [ ]:
# features_transfo = [ 'BILL_AMT1', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
# 
# for feature in features_transfo:
#     df[feature] = df[feature].apply(lambda x: np.sign(x)*np.log(1+np.abs(x)))
# 
# df[features_transfo].head()

### Suppression de variables inutiles, redondantes ou fortement corrélées

In [ ]:
features_to_remove = ['BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']

df = df.drop(features_to_remove, axis=1)

### Regroupement de modalités

In [ ]:
# EDUCATION 0,4,5,6 ==> 0
# MARRIAGE 0,3 ==> AUTRE
# PAY_0 > 4 ==> 0
# PAY_2 = 1,4,5,6,7,8 ==> 0
# PAY_3 = 1,4,5,6,7,8 ==> 0
# PAY_4 = 1,4,5,6,7,8 ==> 0
# PAY_5 = 4,5,6,7,8 ==> 0
# PAY_6 = 4,5,6,7,8 ==> 0

df.loc[df['EDUCATION']== 0,'EDUCATION'] = 2
df.loc[df['EDUCATION'].isin([4,5,6]),'EDUCATION'] = 4

df.loc[df['MARRIAGE'].isin([0,3]), 'MARRIAGE'] = 2
df.loc[df['PAY_0'].isin([5,6,7,8]),'PAY_0'] = 0
df.loc[df['PAY_2'].isin([1,4,5,6,7,8]),'PAY_2'] = 0
df.loc[df['PAY_3'].isin([1,4,5,6,7,8]),'PAY_3'] = 0
df.loc[df['PAY_4'].isin([1,4,5,6,7,8]),'PAY_4'] = 0
df.loc[df['PAY_5'].isin([4,5,6,7,8]),'PAY_5'] = 0
df.loc[df['PAY_6'].isin([4,5,6,7,8]),'PAY_6'] = 0

## Discrétisation des variables numériques

In [ ]:
# features_to_discretize = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
# 
# for feature in features_to_discretize:
#     df[feature] = pd.qcut(df[feature], 5, duplicates='drop', labels=False)
#     df[feature] = df[feature].astype('category')
#     
# df[features_to_discretize].head()

## Création de nouvelles variables

Ici on peut lire et ajouter des embeddings FastText et créer des combinaisons de variables

### Interactions de variables

In [ ]:
df['BILL_AMT1_DIV_LIMIT_BAL'] = df['BILL_AMT1'] / df['LIMIT_BAL']

In [ ]:
df.PAY_AMT2_DIV_BILL_AMT1.hist()

## Séparation train et test

In [ ]:
np.random.seed(123456)
X_train_train, X_train_test = np.split(df.sample(frac = 1), [20000])
print(len(X_train_train))
print(len(X_train_test))

In [ ]:
print(X_train_train[target].value_counts() / len(X_train_train))
print(X_train_test[target].value_counts() / len(X_train_test))

### Exclusion d'outliers sur X_train_train

In [ ]:
from sklearn.covariance import EllipticEnvelope
from scipy import stats

features_for_outlier_detection = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

outliers_fraction = 0.10
clf = EllipticEnvelope(contamination=outliers_fraction)
clf.fit(X_train_train[features_for_outlier_detection])
scores_pred = clf.decision_function(X_train_train[features_for_outlier_detection])
y_pred = clf.predict(X_train_train[features_for_outlier_detection])
threshold = stats.scoreatpercentile(scores_pred, 100 * outliers_fraction)


### Analyse des outliers

In [ ]:
df_outlier = X_train_train[y_pred == -1]

print(df_outlier[target].value_counts() / len(df_outlier))

In [ ]:
X_train_train = X_train_train[y_pred == 1 ]
X_train_train.head()

## Estimation du modèle

In [ ]:
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler, PolynomialFeatures, OneHotEncoder, LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from xgboost.sklearn import XGBClassifier

subset = df.columns[df.columns != target]

# Commenter pour utiliser toutes les features

# subset = best_subset

y = X_train_train[target]
X = X_train_train[subset]

scaler = RobustScaler()

clf = XGBClassifier(n_estimators = 200, max_depth = 3, reg_alpha = 0.001)

scale_gbm = Pipeline([
    ('scale', scaler),
    ('gbm', clf)])


scores = cross_val_score(scale_gbm, X, y, cv=5, scoring= 'roc_auc')

scale_gbm.fit(X=X, y=y)

print(scores)
print('AUC moyen : {}\nstd AUC : {}\nGini moyen : {}'.format(np.average(scores), np.std(scores), 2*np.average(scores) -1 ))


In [ ]:
importances = scale_gbm.named_steps.gbm.feature_importances_
indices = np.argsort(importances)[::-1]

f, ax = plt.subplots(figsize=(6, 15))
importance_df = pd.DataFrame({'feature' : subset[indices],'imp' : importances[indices]})
sns.barplot(x='imp', y='feature', data= importance_df)
sns.despine(left=True, bottom=True)

best_subset = subset[indices][:15]

In [ ]:
from sklearn.metrics import roc_auc_score
   
    
def evaluate_delta(model, test_df, features, target_feature, iterations = 100):
    # This function evaluates the Gini variation according to the challenge rules
    # The test set is divided into private (60%) and public (40%) 
    
    # we will try to evaluate through multiple iterations
    # return a list of length iterations containing the absolute value of deltas
    
    deltas = []
    ginis_public  = []
    ginis_private  = []
    
    for i in range(iterations):
        
        # Split
        public_test, private_test = np.split(test_df.sample(frac = 1), [int(len(test_df)*.4)])
        
        # Prediction on public
        probas_public = model.predict_proba(public_test[features])
        actual_public = public_test[target_feature]
        
        # Prediction on private
        probas_private = model.predict_proba(private_test[features])
        actual_private = private_test[target_feature]
        
        # Gini estimation
        gini_public = 2*roc_auc_score(actual_public, probas_public[:,1]) - 1
        gini_private = 2*roc_auc_score(actual_private, probas_private[:,1]) - 1
        
        ginis_public.append(gini_public)
        ginis_private.append(gini_private)
        
        # Delta 
        delta = (gini_private / gini_public)-1
        deltas.append(np.abs(delta))
    
    return deltas, ginis_public, ginis_private
    

deltas, ginis_public, ginis_private = evaluate_delta(scale_gbm, X_train_test, subset, target, iterations=100)  

print('Gini public moyen : {}'.format(np.mean(ginis_public)))
print('Gini private moyen : {}'.format(np.mean(ginis_private)))
print('delta moyen : {} - delta std : {}'.format(np.mean(deltas), np.std(deltas)))

In [ ]:
def evaluate_lift_matrix(model, test_df, features, target, quantiles = 10):
    
    proba_test = model.predict_proba(test_df[subset])[:,1]
    
    df_probas = pd.DataFrame({
        'proba':proba_test, 
        'target':X_train_test[target]})
    
    df_probas['decile'] = pd.qcut(df_probas['proba'], quantiles, labels = False)
    
    decile_size = df_probas.groupby('decile').size()
    decile_min  = df_probas.groupby('decile')['proba'].min()
    decile_max  = df_probas.groupby('decile')['proba'].max()
    decile_sum_target = df_probas.groupby('decile')['target'].sum()
    
    df_results = pd.DataFrame({
        'size' : decile_size,
        'min' : decile_min,
        'max' : decile_max,
        'sum_target' : decile_sum_target})
        
    df_results = df_results.assign(ratio = lambda df: df['sum_target'] / df['size'])
    
    return df_results


lift_matrix = evaluate_lift_matrix(scale_gbm, X_train_test, subset, target)

f, ax = plt.subplots(figsize=(12, 12))
sns.barplot(x=lift_matrix.index[::-1], y=lift_matrix.ratio)
overall_ratio = X_train_test[target].sum() / len(X_train_test)
plt.axhline(y=overall_ratio, color='r', linestyle='--')
plt.plot()
lift_matrix



## Persistence

On sauvegarde dans un dossier 'save':
- X_train_train
- X_train_testpublic
- X_train_testpriv
- le modèle
- subset
- les résultats

In [ ]:
save_path = 'save'



In [ ]:
df.to_pickle('save/foo.pkl')